In [1]:
import os
import sys
WORKING_DIR = '/data/GitHub/2019/EPA_PSM_antelope'
sys.path.append(WORKING_DIR)
CAT_DIR = os.path.join(WORKING_DIR, 'catalog')

In [2]:
from antelope_epa.epa_psm_antelope import EpaF18Foreground

In [3]:
from antelope_catalog import LcCatalog

In [4]:
cat = LcCatalog(CAT_DIR)

Loading JSON data from /data/GitHub/2019/EPA_PSM_antelope/catalog/reference-quantities.json:
local.qdb: /data/GitHub/2019/EPA_PSM_antelope/catalog/reference-quantities.json
local.qdb: /data/GitHub/lca-tools/lcatools/qdb/data/elcd_reference_quantities.json
25 new quantity entities added (25 total)
6 new flow entities added (6 total)


In [5]:
fg = cat.foreground('psm-hackathon')

local.data.GitHub.2019.EPA_PSM_antelope.catalog.psm-hackathon: /data/GitHub/2019/EPA_PSM_antelope/catalog/psm-hackathon
Loading /data/GitHub/2019/EPA_PSM_antelope/catalog/psm-hackathon
0 new fragment entities added (0 total)


In [6]:
efg = EpaF18Foreground(fg, folder='/data/GitHub/2019/LCAproductsystemassembly_1/data-foreground/')

Workbook: NLG F18EFG ABL.xlsx
Assemblies: ['74A430500-1017', '74A430700-1009', '74A430800-1021']
74A430500-1017: set()
74A430700-1009: set()
74A430800-1021: set()
Missing: set()

Workbook: MLG F18EFG ABL_c.xlsx
Assemblies: ['74A400500-1017', '74A400500-1018', '74A400700-1013', '74A400700-1014', '74A400800-1001']
74A400500-1017: set()
74A400500-1018: set()
74A400700-1013: set()
74A400700-1014: set()
74A400800-1001: set()
Missing: set()

Workbook: MLG F18EFG ABL.xlsx
Assemblies: ['74A400500-1017', '74A400500-1018', '74A400700-1013', '74A400700-1014', '74A400800-1001']
74A400500-1017: {'74A400600-1017', '74A400631-2003'}
74A400500-1018: {'74A400600-1017', '74A400631-2003'}
74A400700-1013: {'74A400700-2009', '74A400700-1011', '74A400780-1001'}
74A400700-1014: {'74A400700-2009', '74A400700-1012', '74A400780-1001'}
74A400800-1001: set()
Missing: {'74A400631-2003', '74A400700-1011', '74A400700-2009', '74A400600-1017', '74A400700-1012', '74A400780-1001'}

Cross-References: set()
Globally Missi

In [7]:
for x in efg.valid_sheets:
    efg.create_assembly(x)

In [8]:
[k.external_ref for k in efg._refs]

['74A430500-1017 ASSEMBLY',
 '74A430700-1009 ASSEMBLY',
 '74A430800-1021 ASSEMBLY',
 '74A400500-1017 ASSEMBLY',
 '74A400500-1018 ASSEMBLY',
 '74A400700-1013 ASSEMBLY',
 '74A400700-1014 ASSEMBLY',
 '74A400800-1001 ASSEMBLY']

In [9]:
efg.fg.count('fragment')

1528

In [10]:
dup_flows = [list(set(k.flow for k in d))[0] for d in efg.duplicate_subassemblies()]

In [11]:
[(k.external_ref, k.name) for k in dup_flows]

[('74A400734-1003', 'UNIV ASSY'),
 ('74B400977-101', 'BEARING'),
 ('74A400730-1003', 'SIDE BR ASSY,UPR'),
 ('74A400939-1001', 'ROD ASSY, SPR'),
 ('74A400780-1003', 'CONN LINK ASSY'),
 ('74A400912-1003', 'BOLT, ASSY, RETNR'),
 ('74A400600-1019', 'SHOCK ABS ASSY'),
 ('74B400914-105', 'BEARING,PLAIN'),
 ('74B400516-2001', 'SCREW, CAP'),
 ('74A400701-1005', 'BELLCRANK ASSY-UPR'),
 ('74A410518-2005A', 'RING, TIEDOWN'),
 ('74A400750-1005', 'LINK ASSY, LWR'),
 ('74A400574-1001', 'LINK ASSY, UPPER'),
 ('74A400561-1005', 'LINK ASSY, LOWER'),
 ('74A400711-1001', 'RING ASSY'),
 ('74A400740-1001', 'SIDE BR ASSY,LWR'),
 ('74A400940-1003', 'AXLE ASSY'),
 ('74A400736-1001', 'ADAPTER ASSY'),
 ('74A400517-1003', 'BOLT ASSY')]

In [12]:
len(dup_flows)

19

In [13]:
_dag = efg.duplicate_subassemblies()

In [14]:
dup_set = list(next(_dag))

In [15]:
dup_set

## check their inventories for identity
We can't use `inventory()` because that runs the inventory of the top (reference) fragment, and neither of these are reference fragments. But we can use the direct traversal and group the inputs and outputs

In [16]:
from lcatools.fragment_flows import group_ios

In [17]:
io0, _ = group_ios(dup_set[0], dup_set[0].traverse(None))

In [18]:
io1, _ = group_ios(dup_set[0], dup_set[1].traverse(None))  # parent is used just for building the exchanges

In [19]:
[str(k) for k in io0]

['74A40           1 [Output] --: 74A400700-1013 ASSEMBLY',
 '74A40           1 [ Input] --: PIN, STRAIGHT',
 'NAS51           2 [ Input] --: FITTING',
 '74A40           2 [ Input] --: BUSHING',
 '74A40           1 [ Input] --: SPINDLE',
 '74A40           1 [ Input] --: PAD, JACK']

In [20]:
[str(k) for k in io1]

['74A40           1 [Output] --: 74A400700-1013 ASSEMBLY',
 '74A40           1 [ Input] --: PAD, JACK',
 '74A40           2 [ Input] --: BUSHING',
 '74A40           1 [ Input] --: SPINDLE',
 'NAS51           2 [ Input] --: FITTING',
 '74A40           1 [ Input] --: PIN, STRAIGHT']

In [21]:
{(k.fragment.flow, k.fragment.direction, k.magnitude) for k in io0} == \
{(k.fragment.flow, k.fragment.direction, k.magnitude) for k in io1}

True

In [ ]:
# store the inventories to test equivalency later

In [22]:
tops = [k.top() for k in dup_set]

In [23]:
u0 = tops[0].inventory()
u1 = tops[1].inventory()

In [ ]:
# now swap them out

In [24]:
efg.reduce_duplicates(*dup_set)

In [25]:
dup_set[0].show()

( ** ref) -<- eeb00 -<- -O    [   1 Item(s)] UNIV ASSY {74A400734-1003 ASSEMBLY}
Fragment Entity (ref 74A400734-1003 ASSEMBLY)
origin: local.data.GitHub.2019.EPA_PSM_antelope.catalog.psm-hackathon
reference: None
     Name: UNIV ASSY
  Comment: 
      Row: 40
StageName: 
   Source: 74A400700-1013
Exchange values: 
              Cached: 1
            Observed: 1

Balance flow: False
Terminations: 
            Scenario  Termination
                None: -O   Foreground


In [26]:
u0post = tops[0].inventory()
{(x.flow, x.direction, x.value) for x in u0} == {(x.flow, x.direction, x.value) for x in u0post}

True

In [27]:
u1post = tops[1].inventory()
{(x.flow, x.direction, x.value) for x in u1} == {(x.flow, x.direction, x.value) for x in u1post}

True

In [28]:
[k.external_ref for k in efg._refs]

['74A430500-1017 ASSEMBLY',
 '74A430700-1009 ASSEMBLY',
 '74A430800-1021 ASSEMBLY',
 '74A400500-1017 ASSEMBLY',
 '74A400500-1018 ASSEMBLY',
 '74A400700-1013 ASSEMBLY',
 '74A400700-1014 ASSEMBLY',
 '74A400800-1001 ASSEMBLY',
 '74A400734-1003 ASSEMBLY']

In [29]:
for dg in _dag:
    efg.reduce_duplicates(*dg)

### How'd we do?
Reduced the count of fragments by.. 8-9%

Do the inventories match?

In [30]:
efg.fg.count('fragment')

1402

In [31]:
u0final = tops[0].inventory()
{(x.flow, x.direction, x.value) for x in u0} == {(x.flow, x.direction, x.value) for x in u0final}

True

In [32]:
u1final = tops[1].inventory()
{(x.flow, x.direction, x.value) for x in u1} == {(x.flow, x.direction, x.value) for x in u1final}

True

In [33]:
# subfragment terminations are shown by '#::'
tops[1].show_tree()

   -<--O   bb47d [       1 Item(s)] 74A400700-1014 ASSEMBLY
    [   1 unit] 74A400700-1014 ASSEMBLY
       | -<----: 168c7 (       1 Item(s)) WASHER, FLAT
       | -<----: 50cb9 (       1 Item(s)) PIN
       | -<----: 5b204 (       1 Item(s)) NUT, PLAIN
       | -<----: 69024 (       1 Item(s)) PLATE, ID
       | -<----: 89ed5 (       1 Item(s)) PIN
       | -<----: b323b (       1 Item(s)) PIN ECCENTRIC
       | -<----: c44a3 (       3 Item(s)) WASHER, FLAT
       | -<----: df9d1 (       1 Item(s)) PLATE, LOCK
       | -<----: e6f67 (       2 Item(s)) NUT, SELF LOCKING
       | -<----: e8f52 (       2 Item(s)) BOLT, SHOULDER
       | -<----: efaed (       1 Item(s)) NUT
       | -<----: f0d6c (       1 Item(s)) BOLT
       | -<--O   090ce (       1 Item(s)) BRACE ASSY
       |  [   1 unit] BRACE ASSY
       |     | -<----: 07988 (       1 Item(s)) SPACER, PLATE
       |     | -<----: 08a5d (       1 Item(s)) BOLT
       |     | -<----: 09264 (       1 Item(s)) NUT, UNIV ATCH
       | 

In [35]:
efg.fg.save()

local.data.GitHub.2019.EPA_PSM_antelope.catalog.psm-hackathon: /data/GitHub/2019/EPA_PSM_antelope/catalog/psm-hackathon/entities.json


In [36]:
new_dups = [k for k in efg.duplicate_subassemblies()]

In [37]:
new_dups

[]